In [5]:
using PyCall
nx = pyimport("networkx")
np = pyimport("numpy")

using IterTools
using Eirene

In [6]:
graph=nx.cycle_graph(8)
graph.add_edge(2,6)
k=2
l=3

3

In [7]:
vtx=collect(graph.nodes())
println(vtx)

Any[0, 1, 2, 3, 4, 5, 6, 7]


In [8]:
MC_kl = Array{Int64}[] 
for possible_chain in collect(Iterators.product([vtx for i=1:k+1]...))
    possible_chain = collect(possible_chain)   
    is_seq = true
    for i in 1:k
        if possible_chain[i] == possible_chain[i+1]
            is_seq = false
        end  
    end
    if is_seq == true   
        length = 0
        for i in 1:k
            length = length + nx.shortest_path_length(graph, possible_chain[i], possible_chain[i + 1])
            if length > l
                break
            end
        end
        if length == l
            push!(MC_kl,possible_chain)
        end
    end                                 
end
sort!(MC_kl)
println(MC_kl)
println(length(MC_kl))

Array{Int64,N} where N[[0, 1, 3], [0, 1, 6], [0, 1, 7], [0, 2, 1], [0, 2, 3], [0, 2, 6], [0, 6, 2], [0, 6, 5], [0, 6, 7], [0, 7, 1], [0, 7, 2], [0, 7, 5], [1, 0, 2], [1, 0, 6], [1, 2, 0], [1, 2, 4], [1, 2, 5], [1, 2, 7], [1, 3, 2], [1, 3, 4], [1, 6, 2], [1, 6, 5], [1, 6, 7], [1, 7, 0], [1, 7, 6], [2, 0, 1], [2, 0, 7], [2, 1, 3], [2, 1, 6], [2, 1, 7], [2, 3, 1], [2, 3, 5], [2, 3, 6], [2, 4, 3], [2, 4, 5], [2, 5, 4], [2, 5, 6], [2, 6, 0], [2, 6, 1], [2, 6, 3], [2, 6, 4], [2, 7, 0], [2, 7, 6], [3, 1, 0], [3, 1, 2], [3, 2, 0], [3, 2, 4], [3, 2, 5], [3, 2, 7], [3, 4, 2], [3, 4, 6], [3, 5, 4], [3, 5, 6], [3, 6, 2], [3, 6, 5], [3, 6, 7], [4, 2, 1], [4, 2, 3], [4, 2, 6], [4, 3, 1], [4, 3, 5], [4, 3, 6], [4, 5, 2], [4, 5, 3], [4, 5, 7], [4, 6, 2], [4, 6, 5], [4, 6, 7], [5, 2, 1], [5, 2, 3], [5, 2, 6], [5, 3, 2], [5, 3, 4], [5, 4, 2], [5, 4, 6], [5, 6, 0], [5, 6, 1], [5, 6, 3], [5, 6, 4], [5, 7, 0], [5, 7, 6], [6, 0, 1], [6, 0, 7], [6, 1, 0], [6, 1, 2], [6, 2, 0], [6, 2, 4], [6, 2, 5], [6, 2, 7]

In [9]:
MC_k_1l = Array{Int64}[] 
for possible_chain in collect(Iterators.product([vtx for i=1:k]...))
    possible_chain = collect(possible_chain)    
    is_seq = true
    for i in 1:k-1
        if possible_chain[i] == possible_chain[i + 1]
            is_seq = false
        end
    end
    if is_seq ==true
        length = 0
        for i in 1:k-1
            length = length + nx.shortest_path_length(graph, possible_chain[i], possible_chain[i + 1])
            if length > l
                break
            end
        end
        if length == l
            push!(MC_k_1l,possible_chain)
        end
    end
end            
sort!(MC_k_1l)
println(MC_k_1l)
println(length(MC_k_1l))

Array{Int64,N} where N[[0, 3], [0, 5], [1, 4], [1, 5], [3, 0], [3, 7], [4, 1], [4, 7], [5, 0], [5, 1], [7, 3], [7, 4]]
12


In [10]:
if length(MC_k_1l)==0
    bdry_mtx = zeros((1, length(MC_kl)))
elseif length(MC_kl)==0
    bdry_mtx = zeros((length(MC_k_1l), 1))
else
    bdry_mtx = zeros((length(MC_k_1l), length(MC_kl)))
end

size(bdry_mtx)

(12, 112)

In [11]:
for k_chain_idx in 1:length(MC_kl)
    k_chain = MC_kl[k_chain_idx]
    #println(k_chain)
    for vertex_idx in 2:length(k_chain) - 1
        #if removing a vertex does not change the length of a path
        if nx.shortest_path_length(graph, k_chain[vertex_idx - 1], k_chain[vertex_idx + 1]) == nx.shortest_path_length(graph,k_chain[vertex_idx - 1],k_chain[vertex_idx]) + nx.shortest_path_length(graph, k_chain[vertex_idx], k_chain[vertex_idx + 1])
            #if the k-tuple with the vertex removed is part of MC_{k-1,l}
            kminus1_chain = collect(np.delete(k_chain, vertex_idx-1))
            #println(kminus1_chain)
            if kminus1_chain in MC_k_1l
                #println(kminus1_chain)
                #set the matrix entry to be -1
                row_index = findfirst(x->x==kminus1_chain,MC_k_1l)
                #println("want to change row:", row_index)
                #println("want to change clmn:", k_chain_idx)
                bdry_mtx[row_index, k_chain_idx] = (-1)^vertex_idx
                #replace(bdry_mtx, bdry_mtx[row_index, k_chain_idx] => -1)
            end
        end
    end
end
bdry_mtx[4,17] 

1.0

In [12]:
chain = MC_kl[4] 

3-element Array{Int64,1}:
 0
 2
 1

In [13]:
np.delete(chain, 1)

2-element Array{Int64,1}:
 0
 1

In [15]:
dv = Vector{Int}(undef,length(MC_k_1l)+length(MC_kl))
println(dv)

[139879131438608, 139879131438656, 139879131438704, 139879131438752, 139879131438800, 139879131438848, 139879131438896, 139879131438944, 139879131438992, 139879131439040, 139879131439088, 139879131439136, 139879131439184, 139879131439232, 139879131439280, 139879131439328, 139879131439376, 139879131439520, 139879131439568, 139879131439616, 139879131439664, 139879131439712, 139879131439760, 139879131439808, 139879131439904, 139879131439952, 139879131440000, 139879131440048, 139879131440096, 139879131440144, 139879131440192, 139879131440240, 139879131440288, 139879131440336, 139879131440384, 139879131440432, 139879131440480, 139879131440528, 139879131440576, 139879131440624, 139879131440672, 139879131440720, 139879131440768, 139879131440816, 139879131440864, 139879131440912, 139879131440960, 139879131441008, 139879131441056, 139879131441104, 139879131441152, 139879131441248, 139879131441296, 139879131441344, 139879131441392, 139879131441440, 139879131441488, 139879131441536, 1398791314416

In [21]:
k=2
for i in 1:length(MC_k_1l)
    dv[i]=k
end
for i in length(MC_k_1l):length(MC_kl)
    dv[i]=k+1
end

In [30]:
dv_k = (k+1)*ones(1,length(MC_k_1l)+length(MC_kl))

1×124 Array{Float64,2}:
 3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  …  3.0  3.0  3.0  3.0  3.0  3.0  3.0

In [31]:
for i in 1:length(MC_k_1l)
    dv_k[i]=k
end

In [32]:
dv_k

1×124 Array{Float64,2}:
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  3.0  3.0  3.0  3.0  3.0  3.0  3.0

In [33]:
B = [5 10; 15 20]
v = size(B)
println(v[1]+v[2])

4
